# SVR Players Data Stats

This notebook loads the shared Google Sheet, cleans the data, computes per-player aggregates (average damage, average kills, matches), and creates comparison plots (violin plot for distributions and line plots for time-series trends).

Notes: the Google Sheet is loaded directly via its CSV export URL. If your sheet has multiple tabs, change the `gid` parameter in the URL.


In [ ]:
# Setup and imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from IPython.display import display

sns.set(style="whitegrid", rc={"figure.dpi": 120})
%matplotlib inline


In [ ]:
# Configuration: sheet id and (optional) gid for sheet tab
SHEET_ID = "19sLvaX2H78wUgxiwjyML-fRUfXYHJOJ7FkrHVlDQv7E"
GID = 0  # change if your data is on a different tab
CSV_URL = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={GID}"
OUT_DIR = 'assets/projects/svr-players-data-stats'
os.makedirs(OUT_DIR, exist_ok=True)

print('Loading from:', CSV_URL)
df = pd.read_csv(CSV_URL)
display(df.head())
print()
print('Data shape:', df.shape)

In [ ]:
# Helper: find likely column names and normalize them
def find_column(df, keywords):
    cols = df.columns.tolist()
    lower = {c:c.lower() for c in cols}
    for kw in keywords:
        for c,l in lower.items():
            if kw in l:
                return c
    return None

# Common expected columns (adjust if needed)
player_col = find_column(df, ['player','name','playername','ign'])
damage_col = find_column(df, ['damage','dmg'])
kills_col = find_column(df, ['kill','kills','kda'])
match_col = find_column(df, ['match','matchid','game','round'])
date_col = find_column(df, ['date','day'])

print('Detected columns:')
print('player ->', player_col)
print('damage ->', damage_col)
print('kills ->', kills_col)
print('match ->', match_col)
print('date ->', date_col)


In [ ]:
# Make a working copy and coerce numeric columns
data = df.copy()
if damage_col: data[damage_col] = pd.to_numeric(data[damage_col], errors='coerce')
if kills_col: data[kills_col] = pd.to_numeric(data[kills_col], errors='coerce')

# Create a canonical column naming for later code
canonical = {}
if player_col: canonical['player'] = player_col
if damage_col: canonical['damage'] = damage_col
if kills_col: canonical['kills'] = kills_col
if match_col: canonical['match'] = match_col
if date_col: canonical['date'] = date_col

data = data.rename(columns={v:k for k,v in canonical.items()})
display(data.head())
print('Canonical columns used:', list(canonical.keys()))


In [ ]:
# Compute per-player aggregates
grouped = data.groupby('player').agg(
    matches=( 'match', 'nunique') if 'match' in data.columns else ( 'player', 'count'),
    mean_damage=( 'damage', 'mean') if 'damage' in data.columns else ( 'player', 'count'),
    mean_kills=( 'kills', 'mean') if 'kills' in data.columns else ( 'player', 'count')
).reset_index()

# Defensive: if columns didn't exist, try safe computations
if 'matches' in grouped.columns and grouped['matches'].isnull().all():
    grouped['matches'] = data.groupby('player').size().values

# Add per-match metrics
if 'mean_damage' in grouped.columns and 'matches' in grouped.columns:
    grouped['damage_per_match'] = grouped['mean_damage']
if 'mean_kills' in grouped.columns and 'matches' in grouped.columns:
    grouped['kills_per_match'] = grouped['mean_kills']

grouped = grouped.sort_values(by='mean_damage', ascending=False).reset_index(drop=True)
display(grouped.head(20))


In [ ]:
# Violin plot: damage distribution for top N players
top_n = 8
top_players = grouped.head(top_n)['player'].tolist()
plot_df = data[data['player'].isin(top_players)].copy()

plt.figure(figsize=(10,6))
sns.violinplot(x='player', y='damage', data=plot_df, order=top_players, inner='quartile', palette='crest')
plt.title(f'Damage distribution for top {top_n} players')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, 'violin_damage_top_players.png'))
plt.show()


In [ ]:
# Line plot: trend of mean damage per match for selected players if match ordering exists
if 'match' in data.columns:
    # ensure a sensible order by match or date
    if 'date' in data.columns:
        data['date'] = pd.to_datetime(data['date'], errors='coerce')
        order_col = 'date'
    else:
        order_col = 'match'
    trend = data.groupby([order_col, 'player']).agg(mean_damage=('damage','mean')).reset_index()
    # pick top 4 players for clarity
    players = top_players[:4]
    plt.figure(figsize=(10,6))
    for p in players:
        tmp = trend[trend['player']==p].sort_values(order_col)
        plt.plot(tmp[order_col], tmp['mean_damage'], marker='o', label=p)
    plt.legend()
    plt.title('Mean damage per match over time (top players)')
    plt.xlabel(order_col)
    plt.ylabel('Mean Damage')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, 'line_mean_damage_top_players.png'))
    plt.show()
else:
    print('No match/date column detected; skipping trend line plot.')


In [ ]:
# Save cleaned data and grouped summary for later use
clean_path = os.path.join(OUT_DIR, 'cleaned_data.csv')
grouped_path = os.path.join(OUT_DIR, 'player_summary.csv')
data.to_csv(clean_path, index=False)
grouped.to_csv(grouped_path, index=False)
print('Saved cleaned data to', clean_path)
print('Saved player summary to', grouped_path)


## Conclusions & Next steps

- Review the generated plots in `assets/projects/svr-players-data-stats/` after running the notebook.
- Adjust `GID` if your data is not in the first tab.
- If you want, I can run the notebook here and commit the generated plots/cleaned CSV to the repo, or you can run it in Colab (Open in Colab link provided in the project page).
